# Setup

In [ ]:
#install.packages("odbc")
#install.packages("DBI")
#install.packages("rstudioapi")

library(odbc)
library(dplyr)
library(stringr)
library(tidyr)
library(ggplot2)
library(e0171)

In [ ]:
#install.packages("eeptools") # to calculate ages
#install.packages("apaTables")
#install.packages("tableone")


In [ ]:
system("sudo su -c 'curl https://packages.microsoft.com/config/rhel/7/prod.repo > /etc/yum.repos.d/mssql-release.repo && exit'")
system("sudo yum remove unixODBC-utf16 unixODBC-utf16-devel")
system("sudo ACCEPT_EULA=Y yum install -y msodbcsql17")
system("sudo ACCEPT_EULA=Y yum install -y mssql-tools")
system("echo 'export PATH='$PATH:/opt/mssql-tools/bin'' >> ~/.bashrc")
system("source ~/.bashrc")
system("sudo yum install -y unixODBC-devel")

In [ ]:
file_path <- '/home/ec2-user/SageMaker/db-credentials.txt'
db_creds_df <- read.table(file_path)
db_creds <- db_creds_df$V3
names(db_creds) <- db_creds_df$V1
db_creds <- trimws(db_creds)
#db_creds

In [ ]:
# load database
db <- 'S35'

In [ ]:
connection_string = paste0("DRIVER={ODBC Driver 17 for SQL Server};",
                          "SERVER=", db_creds['host'], ',', db_creds['port'], ';',
                          "DATABASE=", db, ';',
                          "UID=", db_creds['username'], ';',
                          "PWD={", db_creds['password'], "};")

#connection_string

In [ ]:
db_conn <- DBI::dbConnect(odbc::odbc(), .connection_string = connection_string)

# ICD names

Loading ICD names to join to datasets and filter using keywords

In [ ]:
icdnames<-dbGetQuery(db_conn, "select * from S35.dbo.ICDnames;")
icdnames<-icdnames %>% select(-CONCEPT_PATH)

# DEMOGRAPHICS

In [ ]:
#select patient demographics (subsample of 100 pts)
demo<-dbGetQuery(db_conn,"select * from S35.dbo.phqdemographics;")


Selecting only on the Phq-9 which is PRO:ADV0014. PRO:ADV0045 is the PHQ-2. Specificty isn't quite as high as the PHQ9 but there are far more administered.

# Coding the factor levels for Demographic Variables

In [ ]:
demo %>% head()

In [ ]:
library(haven)
library(purrr)
library(apaTables)

In [ ]:
complete.demo<-complete.df %>% as.matrix() %>% as.data.frame() %>% mutate(
    BIRTH_DATE=as_date(BIRTH_DATE),
    AdminDate=as_date(AdminDate),
Age=age_calc(dob = BIRTH_DATE, enddate = AdminDate, units = "years"))  %>% 
mutate(FPL=case_match(CURRENT_FPL_CD,"DEM|FPL:<=100"~"FPL Below 100","DEM|FPL:151-200"~"FPL 151-200","DEM|FPL:UN"~"FPL Unknown","DEM|FPL:>200
"~"FPL Over 200", "DEM|FPL:101-150"~"FPL 101-150"),
 Marital=case_match(MARITAL_STATUS_CD,"DEM|MARITAL:Other"~"Other",
"DEM|MARITAL:Widowed"~"Widowed",
"DEM|MARITAL:Divorced"~"Divorced",
"DEM|MARITAL:Separated"~"Separated",
"DEM|MARITAL:Domestic Partner"~"Domestic Partner",
"DEM|MARITAL:Significant Other"~"Significant Other",
"DEM|MARITAL:Married"~"Married",
"DEM|MARITAL:Single"~"Single",
"DEM|MARITAL:NI"~"Marital NI"),
Gender=case_match(GENDER_CD,"DEM|GENDER:TG"~"Transgender",
"DEM|GENDER:GQ"~"Genderqueer",
"DEM|GENDER:UN"~"Unknown / Missing",
"DEM|GENDER:OT"~"Other",
"DEM|GENDER:M"~"Man",
"DEM|GENDER:W"~"Woman"),
Race=case_match(RACE_CD,"DEM|RACE:02"~"Asian",
"DEM|RACE:06"~"Multiple race",
"DEM|RACE:01"~"American Indian or Alaskan Native",
"DEM|RACE:05"~"White",
"DEM|RACE:03"~"Black or African American",
"DEM|RACE:07"~"Refuse to answer",
"DEM|RACE:04"~"Native Hawaiian or Other Pacific Islander",
"DEM|RACE:OT"~"Other"),
Rural=case_match(RURAL_CD,"DEM|RURAL:Y"~"Yes",
"DEM|RURAL:N"~"No"),
Sex=case_match(SEX_CD,"DEM|SEX:NI"~"Sex No Informatsuion",
"DEM|SEX:F"~"Female",
"DEM|SEX:M"~"Male",
"DEM|SEX:UN"~"Sex Unknown"),
Sexor=case_match(SEXORIENTATION_CD,"DEM|SEXORIENTATION:GA"~"Gay",
"DEM|SEXORIENTATION:SE"~"Something else",
"DEM|SEXORIENTATION:LE"~"Lesbain",
"DEM|SEXORIENTATION:ST"~"Straight",
"DEM|SEXORIENTATION:MU"~"Multiple sexual orientations",
"DEM|SEXORIENTATION:UN"~"Unknown/ Missing",
"DEM|SEXORIENTATION:BI"~"Bisexual",
"DEM|SEXORIENTATION:QU"~"Queer"))%>% 
mutate(across(c(HEALTH_SYSTEM_ID,CURRENT_FPL_CD:SEXORIENTATION_CD,FPL:Sexor),~factor(.)))


# PHQ

# CURRENT WORKING BAND

In [ ]:
phqcomplete<-dbGetQuery(db_conn,"select * from S35.dbo.phqcomplete;")

In [ ]:
## phqcomplete with depression or hyp filter
phqcomplete<-phqcomplete %>% left_join(icdnames)


In [ ]:
#playing with doing vitals in the dataset in one go to avoid merge problems
phqnewcols<-phqcomplete %>%  mutate(NAME_CHAR=
       case_when(
       str_detect(NAME_CHAR,"Depress")~"Depressed",
       str_detect(NAME_CHAR,"Type II|Type 2|Diabetes")~"Diabetes",
       str_detect(NAME_CHAR,"Hyperlipidemia")~"Hyperlipidemia",
       str_detect(NAME_CHAR,"Hypertens")~"Hypertension",
       str_detect(NAME_CHAR,"Obesity")~"Obesity"
       #.default=NAME_CHAR
       ))%>% mutate(Vitals=
      case_when(
       str_detect(CONCEPT_CD,"BMI")~"BMI", 
       str_detect(CONCEPT_CD,"SYS")~"SystolicBP", 
       str_detect(CONCEPT_CD,"DIAS")~"DiastolicBP",
       str_detect(CONCEPT_CD,"PRO:ADV0039")~"Audit",
       str_detect(CONCEPT_CD,"PRO:ADV0045")~"Phq2",
      )) 

In [ ]:
phqcomplete %>% group_by(NAME_CHAR) %>% tally() %>% arrange(desc(n)) %>% write.csv("ConditionTally.csv")

In [ ]:
# ALTERNATIVE VERSION THAT INCLUDES OTHER DIAGNOSES IN THE SAMPLE
phqalldx<-phqcomplete %>%  mutate(NAME_CHAR=
       case_when(
       str_detect(NAME_CHAR,"Depress|Dysthymic")~"Depressed",
       str_detect(NAME_CHAR,"Type II|Type 2|Diabetes")~"Diabetes",
       str_detect(NAME_CHAR,"Hyperlipidemia")~"Hyperlipidemia",
       str_detect(NAME_CHAR,"Hypertens")~"Hypertension",
       str_detect(NAME_CHAR,"Obesity")~"Obesity",
       str_detect(NAME_CHAR,"Adjustment")~"Adjustment",      
       str_detect(NAME_CHAR,"Anxiety|Panic|Agoraphobia")~"Anxiety",
       str_detect(NAME_CHAR,"Posttraumatic|Post-traumatic")~"PTSD",
       str_detect(NAME_CHAR,"Sleep|Insomnia|F51")~"Sleep",
       str_detect(NAME_CHAR,"Bipolar|Manic")~"Bipolar",
       str_detect(NAME_CHAR,"Nicotine|Tobacco")~"Tobacco",
       str_detect(NAME_CHAR,"Opioid|F11|F12|F13|F14|F15|F16|F18|F19")~"SUD",
       str_detect(NAME_CHAR,"Alcohol|F10")~"Alcohol",
       str_detect(NAME_CHAR,"Schiz|Delusional|F23|F24|F28|F29")~"SchizophreniaSpectrum",     
       str_detect(NAME_CHAR,"Eating|F50")~"EatingDisorders"    
       #.default=NAME_CHAR
       ))%>% mutate(Vitals=
      case_when(
       str_detect(CONCEPT_CD,"BMI")~"BMI", 
       str_detect(CONCEPT_CD,"SYS")~"SystolicBP", 
       str_detect(CONCEPT_CD,"DIAS")~"DiastolicBP",
       str_detect(CONCEPT_CD,"PRO:ADV0039")~"Audit",
       str_detect(CONCEPT_CD,"PRO:ADV0045")~"Phq2",
      )) 

In [ ]:
#Save file from above in case the kernal crashes
save(phqalldx,file="phqalldx.RData")

In [ ]:
load('/home/ec2-user/SageMaker/phqalldx.RData')

In [ ]:
# Reducing the size of the dataset for pivoting
phqfewercols<-phqalldx  %>%  filter(str_detect(CONCEPT_CD,"ICD|ADV0014|ADV0045|ADV0039|BMI|SYS|DIAS"))

In [ ]:
phqfewercols %>% nrow() #14921844

In [ ]:
evenphqfewer<-phqfewercols %>% filter(PhqScore>=10|PhqScore<=4) #limiting to phq extremes to run

In [ ]:
evenphqfewer %>% nrow()

In [ ]:
# rearranging data into wide format: the vitals. formerly phqfewer here
phqwide<-phqfewercols %>% unique() %>% mutate(n=1) %>% 
select(n,phqPT,PhqScore,phqEnc,AdminDate,TVAL_CHAR,NAME_CHAR,INSTANCE_NUM,CONCEPT_CD,NVAL_NUM,HEALTH_SYSTEM_ID,PROVIDER_ID,Vitals)%>% pivot_wider(names_from=Vitals,values_from=NVAL_NUM) 

In [ ]:
save(phqwide,file="wide_vitals.RData")

In [ ]:
load('/home/ec2-user/SageMaker/wide_vitals.RData')

In [ ]:
#checking data integrity
phqwide %>% filter(Audit>1)

In [ ]:
# rearranging wide icd codes
phqwide<-phqwide%>% 
select(n,phqPT,PhqScore,phqEnc,AdminDate,NAME_CHAR,INSTANCE_NUM,CONCEPT_CD,HEALTH_SYSTEM_ID,PROVIDER_ID,BMI,SystolicBP,DiastolicBP,Audit,Phq2) %>% pivot_wider(names_from=NAME_CHAR,values_from=n,values_fill=0)

In [ ]:
save(phqwide,file="phqwide_icd.RData")

In [ ]:
# integrity check again
phqwide %>% filter(Depressed==1)

In [ ]:
phqwide %>% colnames()

In [ ]:
load('/home/ec2-user/SageMaker/phqwide_icd.RData')

In [ ]:
# eliminating unlikely entries
phqwide<-phqwide%>% 
mutate(BMI=case_when(BMI>200|BMI<7~NA_real_,.default=BMI)) %>% 
mutate(SystolicBP=case_when(SystolicBP>300|SystolicBP<50~NA_real_,.default=SystolicBP)) %>% 
mutate(DiastolicBP=case_when(DiastolicBP>300|DiastolicBP<40~NA_real_,.default=DiastolicBP))%>%
mutate(Audit=case_when(Audit>40|Audit<0~NA_real_,.default=Audit))

In [ ]:
phqcollapse<-phqwide %>% group_by(phqPT,phqEnc) %>%  fill(everything(), .direction = "up")  %>% 
fill(everything(), .direction = "down") %>% slice(1)

In [ ]:
save(phqcollapse,file="notindivid.RData")

In [ ]:
# checking for duplicates and its all good

#phqcollapse %>% unique() %>%  ungroup() %>% dplyr::summarise(n = dplyr::n(), .by = c(phqPT, PhqScore, phqEnc,INSTANCE_NUM,CONCEPT_CD,AdminDate)) %>% 
#dplyr::filter(n > 1L)

In [ ]:
phqcollapse %>% nrow() #2843429

In [ ]:
# now I also need to to grab the first instance of an encounter/admindate
phqfirst<-phqcollapse %>% group_by(phqPT) %>% slice_min(AdminDate)

Exclude if no bmi available

In [ ]:
phqfirst1<-phqfirst %>% filter(!is.na(BMI))

In [ ]:
phqfirst1 %>% filter(phqPT==45612) %>% select(Obesity:PTSD)

In [ ]:
#for the 303 remaining with duplicate records on the same day, took random selection
phqfirst2<-phqfirst1 %>% group_by(phqPT) %>% slice_sample(n=1)

In [ ]:
phqfirst2 %>% unique() %>%  ungroup() %>% dplyr::summarise(n = dplyr::n(), .by = c(phqPT)) %>% 
dplyr::filter(n > 1L) #no double records remain

In [ ]:
phqcollapse %>% head()

In [ ]:
save(phqfirst2,file="phqfirst.RDS")

In [ ]:
load(file="/home/ec2-user/SageMaker/phqfirst.RDS")

In [ ]:
load(file="/home/ec2-user/SageMaker/complete.RData")

In [ ]:
phqfirst2 %>% filter(Depressed==1)

In [ ]:
complete.demo %>% colnames()

In [ ]:
demomerge<-complete.demo %>% select(phqPT,phqEnc,AdminDate,phqDepressed:Sexor)

THis is real slipshod but I when I loaded the coded demographic data, some of the fields no longer matched the main phq file

In [ ]:
demomerge %>% head()
demomergenum<-demomerge
demomergenum$phqPT<-as.numeric(as.character(demomerge$phqPT))
demomergenum$phqEnc<-as.numeric(as.character(demomerge$phqEnc))
demomergenum$phqDepressed<-as.numeric(as.character(demomerge$phqDepressed))
demomergenum$phqSevereDepressed<-as.numeric(as.character(demomerge$phqSevereDepressed))
demomergenum$DepAny<-as.numeric(as.character(demomerge$DepAny))

In [ ]:
phqfirst2<-phqfirst2 %>% as.matrix() %>% as.data.frame()

In [ ]:
phq.df<-left_join(phqfirst2,demomergenum,by=join_by(phqPT,phqEnc))

In [ ]:
phqfirst2 %>% nrow() #985645
phq.df %>% nrow() #985645

In [ ]:
save(phq.df,file="phqall_demo.RData")

In [ ]:
# temp to get code to run uses complete cases. It's like a 40% reduction in sample :( and it would be better to impute)
completecases.df<-phq.df %>% ungroup() %>% rowwise() %>%  drop_na(HEALTH_SYSTEM_ID:Sexor)
completecases.df %>% nrow()

In [ ]:
phq.df %>% filter(Depressed==1)

In [ ]:
# then get rid of bad values in phq, round some of the weird values and summarise to check
cleanphq.df<-phq.df %>% filter(PhqScore>0 & PhqScore<28)  %>% mutate(PhqScore=round(PhqScore))
cleanphq.df%>% group_by(Depressed) %>% summarise(mean=mean(PhqScore),sd=sd(PhqScore),min=min(PhqScore),max=max(PhqScore))

In [ ]:
providers<-read.csv('/home/ec2-user/SageMaker/providersepped.csv')


In [ ]:
nrow(providers)

In [ ]:
providers %>% group_by(L1) %>% tally() %>% arrange(desc(n))

In [ ]:
phqdfproviders<-cleanphq.df %>% left_join(providers)

In [ ]:
phqdfproviders %>% group_by(L1) %>% tally() %>% arrange(desc(n)) %>% write.csv("L1_count.csv")
phqdfproviders %>% group_by(L2) %>% tally() %>% arrange(desc(n)) %>% write.csv("L2_count.csv")
phqdfproviders %>% group_by(L3) %>% tally() %>% arrange(desc(n)) %>% write.csv("L3_count.csv")

# Rates of dx

In [ ]:
# create new column for the standard cutoff and a severe cutoff and a totally depressed cutoff
phqdfproviders<-phqdfproviders %>% mutate(phqDepressed=case_when(PhqScore>=10~1,PhqScore<10~0),
                                    phqSevereDepressed=case_when(PhqScore>=20~1,PhqScore<20~0),
                                    phqDefDepressed=case_when(PhqScore>=26~1,PhqScore<26~0))

In [ ]:
save(phqdfproviders,file="phqdf_ready.RData")

In [ ]:
#count of Depression from Any Source
table(phqdfproviders$DepAny)  %>% write.csv("EncDepression_Any_Source.csv")

In [ ]:
phqDepTally<-phqdfproviders %>% group_by(phqDepressed) %>% tally()%>% as.data.frame()
phqSevDepTally<-phqdfproviders %>% group_by(phqSevereDepressed) %>% tally() %>% as.data.frame()
phqDefDepTally<-phqdfproviders %>% group_by(phqDefDepressed) %>% tally()%>% as.data.frame()


phqDepTally
phqSevDepTally
phqDefDepTally

In [ ]:
phqdfproviders %>% group_by(Depressed) %>% tally(phqDepressed==0) %>% mutate(percent=n/nrow(complete.df))

63% of patients with Phq had a low PHQ score and were absent a diagnosis in the record. Only 1% had a low score and a diagnosis in the record.

In [ ]:
phqdfproviders %>% group_by(Depressed) %>% tally(phqDepressed) %>% mutate(percent=n/nrow(phqdfproviders))
phqdfproviders %>% group_by(Depressed) %>% tally(phqDepressed) %>% mutate(percent=n/phqDepTally[2,2])

However, 28% of people with a PHQ  scored >10 did not have a diagnosis in the record. Only 6% of people with a PhQ and elevated score had a corresponding diagnosis in the record. Taken as a proportion of those with elevated sxs, only 18.4% had a depression entry in the record. 

In [ ]:
phqdfproviders %>% group_by(Depressed) %>% tally(phqSevereDepressed) %>% mutate(percent=n/nrow(phqdfproviders))
phqdfproviders %>% group_by(Depressed) %>% tally(phqSevereDepressed) %>% mutate(percent=n/phqSevDepTally[2,2])

If we use the cutoff of 20 where we expect high symptom reporting, we see only 2% of patients with severe scores have a corresponding record of Depressed. 6% of people with severe depression indicated by the phq are listed as not depressed. 
Bottom table is for the same proportions taken out of the number of severely depressed. 76% with high scores are categorized as not depressed and only 23% diagnosed as not depressed.

In [ ]:
phqdfproviders %>% group_by(Depressed) %>% tally(phqSevereDepressed) %>% mutate(percent=n/nrow(phqdfproviders))
phqdfproviders %>% group_by(Depressed) %>% tally(phqDefDepressed) %>% mutate(percent=n/phqDefDepTally[2,2])

In [ ]:
Phqscoretable<-table(phqdfproviders$PhqScore)
Depressedtable<-table(phqdfproviders$Depressed)
phqDepressedtable<-table(phqdfproviders$phqDepressed)
phqSevDeptable<-table(phqdfproviders$phqSevereDepressed)
phqDefDeptable<-table(phqdfproviders$phqDefDepressed)

save(Phqscoretable,Depressedtable,phqDepressedtable,phqSevDeptable,phqDefDeptable,file="PhqCounts.RData")



In [ ]:
library(ggplot2)
ggplot(phqdfproviders,aes(x=PhqScore)) +
  stat_bin(aes(y=cumsum(after_stat(count))))

# ML models

### Prep ML model 

In [ ]:
mlsubset<-phqdfproviders %>% slice_sample(n=1000)

In [ ]:
mlsubset<-mutate(mlsubset,Depressed=as.factor(Depressed),DepAny=as.factor(DepAny),phqDepressed=as.factor(phqDepressed),phqSevereDepressed=as.factor(phqSevereDepressed))

In [ ]:
save(mlsubset,file="mlsubset.RData")

In [ ]:
mlsubset %>% table(mlsubset$Depressed)

### SVM Model

In [1]:
load(file="/home/ec2-user/SageMaker/mlsubset.RData")

In [4]:
library(dplyr)
library(e1071)

In [ ]:
ls()

In [5]:
#set seed for random number generation
set.seed(10)
mlsubset<-mlsubset %>% ungroup() %>% select(BMI:phqDefDepressed,-X,-`NA`)
#split train and test data 80/20
mlsubset[,"train"] <- ifelse(runif(nrow(mlsubset))<0.8,1,0)
trainset <- mlsubset[mlsubset$train==1,]
testset <- mlsubset[mlsubset$train==0,]
#find “train” column index
trainColNum <- grep("train",names(trainset))
#remove column from train and test sets
trainset <- trainset[,-trainColNum]
testset <- testset[,-trainColNum]



In [ ]:
trainset %>% colnames()

In [6]:
mlsubset<-mlsubset %>% slice_sample(n=100)

In [8]:
#build default cost model with Admin Date as garbage predictor
library(e1071)
garbageDep.svm<- svm(Depressed~AdminDate.y, data=trainset,type="C-classification", kernel="linear")

In [9]:
summary(garbageDep.svm)


Call:
svm(formula = Depressed ~ AdminDate.y, data = trainset, type = "C-classification", 
    kernel = "linear")


Parameters:
   SVM-Type:  C-classification 
 SVM-Kernel:  linear 
       cost:  1 

Number of Support Vectors:  80260

 ( 40134 40126 )


Number of Classes:  2 

Levels: 
 0 1




In [7]:
#build default cost model with Encounter Diagnoses
svm_model<- svm(Depressed~., data=trainset,type="C-classification", kernel="linear")

In [10]:
summary(svm_model)


Call:
svm(formula = Depressed ~ ., data = trainset, type = "C-classification", 
    kernel = "linear")


Parameters:
   SVM-Type:  C-classification 
 SVM-Kernel:  linear 
       cost:  1 

Number of Support Vectors:  4940

 ( 2799 2141 )


Number of Classes:  2 

Levels: 
 0 1




In [11]:
#training accuracy
pred_train <- predict(svm_model,trainset)
pred_train_garb <- predict(garbageDep.svm,trainset)

In [12]:
mean(pred_train==trainset$Depressed)
mean(pred_train_garb==trainset$Depressed)



Warning message in `==.default`(pred_train, trainset$Depressed):
“longer object length is not a multiple of shorter object length”
Warning message in is.na(e1) | is.na(e2):
“longer object length is not a multiple of shorter object length”


[1] 0.8924324

Warning message in `==.default`(pred_train_garb, trainset$Depressed):
“longer object length is not a multiple of shorter object length”
Warning message in is.na(e1) | is.na(e2):
“longer object length is not a multiple of shorter object length”


[1] 0.9019881

In [ ]:
length(pred_train)
length(trainset$Depressed)

In [ ]:
# Confusion matrix
library(caret)
confusionMatrix(data=pred_train,reference=trainset)

In [ ]:
cm = table(test_set[, 3], pred_train)

In [ ]:
pred_test <- predict(svm_model,testset)
#mean(pred_test==testset$Depressed)


In [ ]:
set.seed(10)
accuracy <- rep(NA,100)
#calculate test accuracy for 100 different partitions
for (i in 1:100){
mlsubset[,"train"] <- ifelse(runif(nrow(mlsubset))<0.8,1,0)
trainColNum <- grep("train",names(mlsubset))
trainset <- mlsubset[mlsubset$train==1,-trainColNum]
testset <- mlsubset[mlsubset$train==0,-trainColNum]
svm_model <- svm(Depressed~ AdminDate, data=trainset, type="C-classification", kernel="linear")
pred_test <- predict(svm_model,testset)
accuracy[i] <- mean(pred_test==testset$Depressed)
}
mean(accuracy)
sd(accuracy)


The models perform too well, even with information that should be basically random like Admin date

## Same as above but trying DepAny

In [ ]:
#build default cost model with DepAny Diagnosis
library(e1071)
svm_model<- svm(DepAny~phqEnc, data=trainset,type="C-classification", kernel="linear")

In [ ]:
svm_model

In [ ]:
#training accuracy
pred_train <- predict(svm_model,trainset)

In [ ]:
mean(pred_train==trainset$DepAny)



In [ ]:
pred_test <- predict(svm_model,testset)
mean(pred_test==testset$DepAny)


In [ ]:

set.seed(10)
accuracy <- rep(NA,100)
#calculate test accuracy for 100 different partitions
for (i in 1:100){
mlsubset[,"train"] <- ifelse(runif(nrow(mlsubset))<0.8,1,0)
trainColNum <- grep("train",names(mlsubset))
trainset <- mlsubset[mlsubset$train==1,-trainColNum]
testset <- mlsubset[mlsubset$train==0,-trainColNum]
svm_model <- svm(DepAny~ AdminDate, data=trainset, type="C-classification", kernel="linear")
pred_test <- predict(svm_model,testset)
accuracy[i] <- mean(pred_test==testset$DepAny)
}
mean(accuracy) #.637
sd(accuracy) #.008



In [ ]:
# test of a full model with features 
mlsubsetSVM<-mlsubset  %>% select(AdminDate,INSTANCE_NUM,HEALTH_SYSTEM_ID,BMI:SEXORIENTATION_CD,DepAny,-`NA`,-train)

In [ ]:
mlsubsetSVM %>% colnames()

In [ ]:
# results of model with no demographic features
set.seed(10)
accuracy <- rep(NA,100)
#calculate test accuracy for 100 different partitions
for (i in 1:100){
mlsubsetSVM[,"train"] <- ifelse(runif(nrow(mlsubsetSVM))<0.8,1,0)
trainColNum <- grep("train",names(mlsubsetSVM))
trainset <- mlsubsetSVM[mlsubsetSVM$train==1,-trainColNum]
testset <- mlsubsetSVM[mlsubsetSVM$train==0,-trainColNum]
svm_model <- svm(DepAny~INSTANCE_NUM+BMI+SystolicBP+DiastolicBP+Diabetes+Obesity+Hyperlipidemia+Hypertension, data=trainset, type="C-classification", kernel="linear")
pred_test <- predict(svm_model,testset)
accuracy[i] <- mean(pred_test==testset$DepAny)
}
mean(accuracy) #.7145
sd(accuracy) #.00922



In [ ]:
#

### Decision Tree

In [ ]:
library(rpart)
#install.packages("rpart.plot")
#install.packages("rattle")

library(rpart.plot)

In [ ]:
Deptree <- rpart(mlsubset$Depressed ~ mlsubset$PhqScore)

In [ ]:
#provider tree with depressed encounter
Deptreeprov <- rpart(mlsubset$Depressed ~ mlsubset$L2+mlsubset$PhqScore)

In [ ]:
#provider tree with depressed any
Deptreeprov <- rpart(mlsubset$DepAny ~ mlsubset$L2+mlsubset$Race)

In [ ]:
Deptreeprov

In [ ]:
rpart.plot(Deptreeprov, cex=.3)

In [ ]:
Depanytree <- rpart(mlsubset$DepAny ~ mlsubset$HOMELESS_CD+mlsubset$HISPANIC_CD+mlsubset$RURAL_CD+mlsubset$CURRENT_FPL_CD+mlsubset$SEXORIENTATION_CD+mlsubset$SEX_CD
                    +mlsubset$RACE_CD+mlsubset$BMI+mlsubset$SystolicBP+mlsubset$DiastolicBP+mlsubset$Obesity+mlsubset$Hyperlipidemia+mlsubset$Hypertension)

In [ ]:
Depanytree

In [ ]:
rpart.plot(Depanytree, cex=.3)

In [ ]:
library(rattle)
fancyRpartPlot(Depanytree, caption = NULL)

pdf(file="depanytree1.pdf")
fancyRpartPlot(Depanytree, caption = NULL,cex=.3)
dev.off()